In [1]:
import pandas as pd
data = pd.read_csv("../Data/gpascore.csv")
data.head()

,admit,gre,gpa,rank
0,0,380.0,3.21,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.33,4


admit : 합격여부  
gre : 영어성적  
gpa : 학점  
rank : 지원한 대학원 수준  

### Preprocessing(전처리)

In [2]:
# 결측치 처리
data.isnull().sum()


admit    0
gre      1
gpa      0
rank     0
dtype: int64

In [3]:
# 결측치 데이터 삭제
data.dropna(inplace=True)
# 결측치 데이터를 다른값으로 대체
# data.fillna(100)

In [4]:
data.isnull().sum()


admit    0
gre      0
gpa      0
rank     0
dtype: int64

In [5]:
# rank의 종류 파악
data['rank'].value_counts()

2    158
3    129
4     71
1     67
Name: rank, dtype: int64

In [6]:
# 영어 점수의 최소값과 최대값
print('최대값 :', data['gre'].max())
print('최소값 :', data['gre'].min())

최대값 : 800.0
최소값 : 220.0


In [7]:
# Training Data
x = data[['gre','gpa','rank']]
x.head()

,gre,gpa,rank
0,380.0,3.21,3
1,660.0,3.67,3
2,800.0,4.00,1
3,640.0,3.19,4
4,520.0,2.33,4


In [8]:
# target Data
y = data['admit']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: admit, dtype: int64

In [9]:
x.count()

gre     425
gpa     425
rank    425
dtype: int64

In [18]:
# target Data
y = data['admit']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: admit, dtype: int64

---
### Deep Learning Model 만들기

In [11]:
from tensorflow import keras

In [15]:
input_layer = keras.layers.InputLayer(input_shape=(3,))
hidden_layer = keras.layers.Dense(256, activation= 'tanh')
output_layer = keras.layers.Dense(1, activation = 'sigmoid')


In [16]:
model = keras.Sequential(
    [
        input_layer,
        hidden_layer,
        output_layer
    ]
)

2022-07-20 10:53:42.888266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# 손실함수
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
# Optimizer : learning rate를 알맞게 조정해줌
# - adam
# - adagrad
# - adadelta
# - rmsprop
# - sgd 경사하강법 ,
# 나머지들도 다 출발은 경사하강법인데 최저값을 넘어갔다가 돌아오고 이런점이 조금씩 다름
# loss : 손실함수
# - binary_crossentropy : 분류 및 확률 예측
# metrics : 모델의 평가 기준

In [37]:
x.shape

(425, 3)

In [28]:
# 학습 시키기
import numpy as np
model.fit(np.array(x),np.array(y),epochs=500) 

#verbose =  는 진행상황이 안나오게 함

Epoch 1/500
14/14 [==============================] - 0s 5ms/step - loss: 0.6488 - accuracy: 0.6094
Epoch 2/500
14/14 [==============================] - 0s 1ms/step - loss: 0.7123 - accuracy: 0.4965
Epoch 3/500
14/14 [==============================] - 0s 1ms/step - loss: 0.6486 - accuracy: 0.6329
Epoch 4/500
14/14 [==============================] - 0s 1ms/step - loss: 0.6419 - accuracy: 0.6306
Epoch 5/500
14/14 [==============================] - 0s 1ms/step - loss: 0.6432 - accuracy: 0.6306
Epoch 6/500
14/14 [==============================] - 0s 1ms/step - loss: 0.6245 - accuracy: 0.6541
Epoch 7/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6320 - accuracy: 0.6376
Epoch 8/500
14/14 [==============================] - 0s 3ms/step - loss: 0.6454 - accuracy: 0.6376
Epoch 9/500
14/14 [==============================] - 0s 2ms/step - loss: 0.6205 - accuracy: 0.6541
Epoch 10/500
14/14 [==============================] - 0s 3ms/step - loss: 0.6174 - accuracy: 0.6800
Epoch 11/

In [51]:
# hidden layer의 가중치 값 출력
hidden_layer.get_weights()[0][0][0]

0.03771671

In [49]:
hidden_layer.get_weights()[1][0] # 는 bias 값

1.2927994e-05

In [56]:
def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x)+np.exp(-x))

In [57]:
np.array(x)[0][1]

3.21

In [58]:
sum = 0
for i in range(3):
    sum += np.array(x)[0][i] *  hidden_layer.get_weights()[0][i][0]

sum+= hidden_layer.get_weights()[1][0]
print(sum,'=======',tanh(sum))


13.904856083363411 ======= 0.9999999999983272


In [59]:
# hidden layer의 출력 확인하기
intermediate_layer_model = keras.Model(inputs= model.input, outputs= model.layers[0].output) # 입력층은 레이어로 안친다.
intermediate_output = intermediate_layer_model(np.array(x))


In [66]:
intermediate_output[0]

<tf.Tensor: shape=(256,), dtype=float32, numpy=
array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        , -1.        ,  1.        ,  1.        , -0.9999687 ,
       -1.        , -1.        ,  1.        ,  1.        ,  1.        ,
       -1.        , -1.        ,  1.        , -1.        , -1.        ,
       -1.        , -1.        , -1.        ,  1.        , -1.        ,
       -1.        , -0.02996846,  1.        ,  1.        ,  0.99999905,
        1.        ,  1.        , -0.9999963 ,  1.        ,  1.        ,
        1.        , -1.        ,  1.        ,  1.        ,  0.13640952,
        1.        ,  1.        , -1.        ,  1.        ,  1.        ,
       -1.        , -1.        , -1.        ,  0.9999286 , -1.        ,
        1.        ,  1.        ,  0.12033966,  1.        , -0.9999997 ,
       -1.        , -1.        , -1.        , -1.        , -1.        ,
       -1.        ,  1.        , -1.        ,  1.        ,  1.        ,
        1.      

In [31]:
# 예측하기

model.predict(
    [
        [700,3.7,1],
        [700,3.7,2],
        [700,3.7,3],
        [700,3.7,4],
    ]
)

1/1 [==============================] - 0s 99ms/step


array([[0.73349446],
       [0.7035557 ],
       [0.6670077 ],
       [0.6230431 ]], dtype=float32)

In [ ]:
# 예측하기

model.predict(
    [
        [400,2.2,1],
        [400,2.2,2],
        [400,2.2,3],
        [400,2.2,4],
    ]
)

1/1 [==============================] - 0s 18ms/step


array([[0.4547993 ],
       [0.4547993 ],
       [0.45479923],
       [0.45479923]], dtype=float32)